In [5]:
import scanpy as sc
import pandas as pd
import scvi

c:\Users\19utk\anaconda3\envs\ds\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
adata = sc.read_10x_h5('Donor2_filtered_feature_bc_matrix.h5')

c:\Users\19utk\anaconda3\envs\ds\Lib\site-packages\anndata\_core\anndata.py:1793: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
c:\Users\19utk\anaconda3\envs\ds\Lib\site-packages\anndata\_core\anndata.py:1793: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [3]:
adata.var_names_make_unique()
adata 

AnnData object with n_obs × n_vars = 13330 × 33538
    var: 'gene_ids', 'feature_types', 'genome'

In [4]:
sc.pp.filter_genes(adata, min_cells=3)

In [6]:
sc.pp.highly_variable_genes(
    adata,
    n_top_genes=2000,      
    flavor='seurat_v3',
    subset=True             # subset adata to HVGs
)

In [7]:
scvi.model.SCVI.setup_anndata(adata)
model = scvi.model.SCVI(adata)
model.train()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
c:\Users\19utk\anaconda3\envs\ds\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:433: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


Epoch 400/400: 100%|██████████| 400/400 [16:39<00:00,  2.70s/it, v_num=1, train_loss_step=646, train_loss_epoch=560]

`Trainer.fit` stopped: `max_epochs=400` reached.


Epoch 400/400: 100%|██████████| 400/400 [16:39<00:00,  2.50s/it, v_num=1, train_loss_step=646, train_loss_epoch=560]


In [8]:
solo = scvi.external.SOLO.from_scvi_model(model)
solo.train()

INFO     Creating doublets, preparing SOLO model.                                                                  


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
c:\Users\19utk\anaconda3\envs\ds\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:433: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.
c:\Users\19utk\anaconda3\envs\ds\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:433: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


Epoch 352/400:  88%|████████▊ | 352/400 [03:58<00:32,  1.48it/s, v_num=1, train_loss_step=0.121, train_loss_epoch=0.339]
Monitored metric validation_loss did not improve in the last 30 records. Best score: 0.311. Signaling Trainer to stop.


In [9]:
df = solo.predict() 
df['prediction'] = solo.predict(soft = False) 
df.head(3)

c:\Users\19utk\anaconda3\envs\ds\Lib\site-packages\torch\utils\_contextlib.py:120: UserWarning: Prior to scvi-tools 1.1.3, `SOLO.predict` with `soft=True` (the default option) returned logits instead of probabilities. This behavior has since been corrected to return probabiltiies. The previous behavior can be replicated by passing in `return_logits=True`.
  return func(*args, **kwargs)
c:\Users\19utk\anaconda3\envs\ds\Lib\site-packages\torch\utils\_contextlib.py:120: UserWarning: Prior to scvi-tools 1.1.3, `SOLO.predict` with `soft=True` (the default option) returned logits instead of probabilities. This behavior has since been corrected to return probabiltiies. The previous behavior can be replicated by passing in `return_logits=True`.
  return func(*args, **kwargs)


,doublet,singlet,prediction
AAACCCAAGACTGAGC-1,0.043889,0.956111,singlet
AAACCCAAGGTTACCT-1,0.053804,0.946196,singlet
AAACCCAAGTGGATAT-1,0.082613,0.917387,singlet


In [10]:
df["revised_prediction"] = df.apply(
    lambda row: "doublet" if row["doublet"] > 0.8 else "singlet", axis=1
)
df.head(2)

,doublet,singlet,prediction,revised_prediction
AAACCCAAGACTGAGC-1,0.043889,0.956111,singlet,singlet
AAACCCAAGGTTACCT-1,0.053804,0.946196,singlet,singlet


In [11]:
df.to_pickle("donor2_solo_out.pkl")